In [1]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-8der17da
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-8der17da
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
# coding : utf-8
import tensorflow as tf
tf.__version__
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from keras.layers import Dense
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler    
from sklearn.metrics import confusion_matrix
import pandas as pd


In [3]:
%%sh
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
Address sizes:       46 bits physical, 48 bits virtual
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:            7
CPU MHz:             3100.222
BogoMIPS:            4999.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movb

In [4]:

columns = ['age','workclass','fnlwgt','education','education-num','marital-status', 'occupation', 'race', 'relationship', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']
data = pd.read_csv('./adult.data.csv', names=columns)


In [5]:
data.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,race,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,label
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [6]:
data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
race              object
relationship      object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
label             object
dtype: object

In [7]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,race,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
#on binairise toutes nos colonnes.
column_to_bin = ['workclass',
                     'education','marital-status', 'occupation', 'race', 'relationship', 'sex', 'native-country']
for c in column_to_bin:
  if data[c].dtype == 'object':
    data[c] = pd.Categorical(data[c]) 
    catfeat = pd.get_dummies(data[c], prefix = c)
    data = data.drop([c], axis = 1)
    data = pd.concat([data, catfeat], axis=1)

In [9]:
data.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'label', 'workclass_ ?', 'workclass_ Federal-gov',
       'workclass_ Local-gov',
       ...
       'native-country_ Portugal', 'native-country_ Puerto-Rico',
       'native-country_ Scotland', 'native-country_ South',
       'native-country_ Taiwan', 'native-country_ Thailand',
       'native-country_ Trinadad&Tobago', 'native-country_ United-States',
       'native-country_ Vietnam', 'native-country_ Yugoslavia'],
      dtype='object', length=109)

In [10]:
data = data.dropna()

In [11]:
data_stats = data.describe().transpose()
data_stats

,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0
education-num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
capital-gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
capital-loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
...,...,...,...,...,...,...,...,...
native-country_ Thailand,32561.0,0.000553,0.023506,0.0,0.0,0.0,0.0,1.0
native-country_ Trinadad&Tobago,32561.0,0.000584,0.024149,0.0,0.0,0.0,0.0,1.0
native-country_ United-States,32561.0,0.895857,0.305451,0.0,1.0,1.0,1.0,1.0
native-country_ Vietnam,32561.0,0.002058,0.045316,0.0,0.0,0.0,0.0,1.0


In [12]:
compare = data['label'][0]
data['label'] = data['label'].apply(lambda x: 0 if x is not compare else 1)
labels = data.pop('label')

In [13]:
data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,...,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,0.056386,0.029483,0.064279,0.000215,...,0.001136,0.003501,0.000369,0.002457,0.001566,0.000553,0.000584,0.895857,0.002058,0.000491
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,0.230670,0.169159,0.245254,0.014661,...,0.033691,0.059068,0.019194,0.049507,0.039546,0.023506,0.024149,0.305451,0.045316,0.022162
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
## How to use Keras and Tensorflow in classifing adult income data in Python
EPOCHS = 200
REPORTSTEP = 50
DOTS = 1

# Split Train and Test Datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33)  

# normalize the dataset i.e. X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)    
X_test = scaler.fit_transform(X_test)       

In [15]:
# Define a Deep Learning Model
model = Sequential([layers.Dense(32, activation='tanh', input_shape=[len(data.keys())]),
                        layers.Dropout(rate=.1),
                        layers.Dense(64, activation='tanh'),
                        layers.Dropout(rate=.1),
                        layers.Dense(16, activation='tanh'),
                        layers.Dropout(rate=.1),
                        layers.Dense(1, activation='sigmoid')
                        ])
optimizer = tf.keras.optimizers.Adam(0.01)


In [16]:
# Compile the Model
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [17]:
# Train the Model
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=100, shuffle=True, verbose = 0,callbacks=[tfdocs.modeling. EpochDots (report_every=REPORTSTEP)])


Epoch: 0, accuracy:0.8272,  loss:0.3769,  
..................................................
Epoch: 50, accuracy:0.8574,  loss:0.3077,  
..................................................
Epoch: 100, accuracy:0.8636,  loss:0.2957,  
..................................................
Epoch: 150, accuracy:0.8661,  loss:0.2903,  
..................................................

In [19]:
# Evaluate the model
scores = model.evaluate(X_test, y_test)
print(model.metrics_names); print(scores)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Confusion Matrix
y_pred = model.predict(X_test); y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm.astype('float')/cm.sum(axis=1)[:, np.newaxis])

# More on the Model
print(model.summary())

336/336 [==============================] - 0s 871us/step - loss: 0.3352 - accuracy: 0.8449

['loss', 'accuracy']
[0.3352479636669159, 0.8448725342750549]

accuracy: 84.49%

Confusion Matrix:
 [[0.63832243 0.36167757]
 [0.0892353  0.9107647 ]]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                3488      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1040      
___________

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=93afa6b5-2b21-4975-b981-4546d071aaf0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>